In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
best_model_path = "models/fr_lat_model4.pth"
target_col = 'Frontal/Lateral'

In [4]:
class CSVDataset(Dataset):
    def __init__(self, dataframe, image_root_dir, target_columns=None, transform=None,
                 save_dir=None, use_saved_images=False):
        self.data = dataframe
        self.image_root_dir = image_root_dir
        self.target_columns = target_columns
        self.transform = transform
        self.save_dir = save_dir
        self.use_saved_images = use_saved_images

        if self.save_dir:
            os.makedirs(self.save_dir, exist_ok=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Use index for the saved tensor filename
        image_index = row['Unnamed: 0']
        saved_image_path = os.path.join(self.save_dir, f"{image_index}.pt")

        if self.use_saved_images:
            if os.path.exists(saved_image_path):
                image_tensor = torch.load(saved_image_path)
            else:
                raise FileNotFoundError(f"Saved tensor not found: {saved_image_path}")
        else:
            original_image_path = os.path.join(self.image_root_dir, row['Path'])
            image = Image.open(original_image_path).convert("L")
            image_tensor = self.transform(image) if self.transform else transforms.ToTensor()(image)

            if self.save_dir:
                torch.save(image_tensor, saved_image_path)

        if self.target_columns:
            labels = pd.to_numeric(row[self.target_columns], errors='coerce').fillna(0).astype(float).values
            labels = torch.tensor(labels, dtype=torch.float32)
            return image_tensor, labels

        return image_tensor

In [5]:
class MultiLabelResNet50(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelResNet50, self).__init__()
        
        # Load pre-trained ResNet50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Modify the fully connected layer for multi-label classification
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 512),  # New intermediate layer. ##512 --> 256
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout to prevent overfitting ##0.5 --> 0.6
            nn.Linear(512, num_classes),  # Output layer
            #nn.Sigmoid()  # Sigmoid for multi-label classification (soften the data)
            #nn.Tanh()  #This is between -1 and 1
        )

    def forward(self, x):
        return self.base_model(x)

In [6]:
import os

def check_missing_pt_files(df, save_dir):
    missing_rows = []

    for idx, row in df.iterrows():
        pt_path = os.path.join(save_dir, f"{int(row['Unnamed: 0'])}.pt")
        if not os.path.exists(pt_path):
            missing_rows.append(row['Unnamed: 0'])  # or idx if you prefer

    return missing_rows

missing_pt_ids = check_missing_pt_files(full_train_df, train_save_dir)
print(f"Missing .pt files for {len(missing_pt_ids)} entries")
print(missing_pt_ids[:10])  # Show first 10 missing IDs


NameError: name 'full_train_df' is not defined

In [6]:
image_root = '/central/groups/CS156b/2025/CodeMonkeys/input_images'
image_root_dir = "input_images/train"

def get_filtered_df(col, num=None):
    full_train_df = pd.read_csv('train2023.csv')
    full_train_df = full_train_df[full_train_df['Unnamed: 0'] != 223413]

    if num is not None:
        full_train_df = full_train_df.iloc[:num]

    filtered_train_df = full_train_df.dropna(subset=[col]).copy()

    # Map 'Frontal' to 0, 'Lateral' to 1 — only if column is 'Frontal/Lateral'
    if col == 'Frontal/Lateral':
        mapping = {'Frontal': 0, 'Lateral': 1}
        filtered_train_df[col] = filtered_train_df[col].map(mapping)

    return filtered_train_df

df = get_filtered_df(target_col)

In [7]:
##DONT GENERALLY WANT THIS
label_counts = df[target_col].value_counts()

# Get majority class (should be 1.0)
majority_class = label_counts.idxmax()

# Separate the majority and minority classes
majority_df = df[df[target_col] == majority_class]
minority_df = df[df[target_col] != majority_class]

# Randomly sample 10,000 from the majority class
majority_df = majority_df.sample(n=25860, random_state=42)
#majority_df = majority_df.sample(n=2000, random_state=42)
#minority_df = minority_df.sample(n=2000, random_state=42)

# Combine back
balanced_df = pd.concat([majority_df, minority_df]).sample(frac=1, random_state=42).reset_index(drop=True)
df = balanced_df


In [8]:
print(df['Frontal/Lateral'].value_counts())

Frontal/Lateral
1    25860
0    25860
Name: count, dtype: int64


In [10]:
from sklearn.model_selection import train_test_split

train_save_dir = os.path.join(image_root, 'train')
full_train_df = pd.read_csv('train2023.csv')

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = CSVDataset(
    dataframe=train_df, 
    image_root_dir=image_root, 
    target_columns=['Frontal/Lateral'], 
    save_dir=train_save_dir, 
    use_saved_images=True)

val_dataset = CSVDataset(
    dataframe=val_df, 
    image_root_dir=image_root, 
    target_columns=['Frontal/Lateral'], 
    save_dir=train_save_dir, 
    use_saved_images=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
def freeze_base_layers(model, until_layer=6):
    """
    Freeze layers of ResNet-50 up to a certain stage (e.g., until_layer=6 means keep layers 0-5 frozen).
    """
    child_counter = 0
    for child in model.base_model.children():
        if child_counter < until_layer:
            for param in child.parameters():
                param.requires_grad = False
        child_counter += 1
    return model

In [31]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()#pos_weight=torch.tensor(152298 / 25860))

num_classes = 1  # Predicting 'Pleural Effusion'
model = MultiLabelResNet50(num_classes=num_classes).to(device)
model = freeze_base_layers(model, until_layer=2)  # Freeze layers
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)


early_stopping_patience = 3
best_val_loss = float('inf')
patience_counter = 0
os.makedirs("models", exist_ok=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)


num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

        probs = torch.sigmoid(outputs)
        predicted_class = torch.where(probs < 0.5, 0.0, 1.0)

        correct += (predicted_class == labels).sum().item()
        total += labels.numel()

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            probs = torch.sigmoid(outputs)
            predicted_class = torch.where(probs < 0.5, 0.0, 1.0)


            all_preds.append(predicted_class.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

            val_correct += (predicted_class == labels).sum().item()
            val_total += labels.numel()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    scheduler.step(avg_val_loss)

    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        #print("✅ Saved new best model.")
    else:
        patience_counter += 1
     #   print(f"⏳ No improvement. Patience: {patience_counter}/{early_stopping_patience}")

    # Stop if patience exceeded
    if patience_counter >= early_stopping_patience:
        print("⛔ Early stopping triggered.")
        break


Epoch [1/30], Loss: 0.0068, Accuracy: 0.9980
Validation Loss: 0.0016, Validation Accuracy: 0.9994


KeyboardInterrupt: 

In [6]:
model = MultiLabelResNet50(num_classes=1).to(device)
model.load_state_dict(torch.load('models/fr_lat_model4.pth'))
model.eval()

MultiLabelResNet50(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [7]:
import os
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

test_dir = "input_images/solution"
frontal_dir = "input_images/solution_frontal"
lateral_dir = "input_images/solution_lateral"
os.makedirs(frontal_dir, exist_ok=True)
os.makedirs(lateral_dir, exist_ok=True)

# Process each .pt test image
file_list = [f for f in os.listdir(test_dir) if f.endswith(".pt")]

for filename in tqdm(file_list):
    path = os.path.join(test_dir, filename)
    image_tensor = torch.load(path).unsqueeze(0).to(device)  # Add batch dim

    with torch.no_grad():
        output = model(image_tensor)
        prob = torch.sigmoid(output)[0, 0].item()
    # Choose folder based on prediction (e.g., use 0.5 as threshold)
    if prob > 0.5:
        dest = os.path.join(lateral_dir, filename)  # Model thinks it's lateral
    else:
        dest = os.path.join(frontal_dir, filename)  # Model thinks it's frontal

    torch.save(image_tensor.squeeze(0).cpu(), dest)  # Remove batch dim before saving



  0%|          | 0/22660 [00:00<?, ?it/s]

100%|██████████| 22660/22660 [20:37<00:00, 18.32it/s]


In [8]:
import os

lateral_dir = 'input_images/train'
lateral_files = os.listdir(lateral_dir)

print(f"Found {len(lateral_files)} files in {lateral_dir}:")
for f in lateral_files[:10]:  # Show first 10 files
    print(f)

# If you want to see all files, remove the [:10] slice


Found 0 files in input_images/train:
